In [1]:
# prompt: fetch this file from githubhttps://github.com/hs-pm/PulseChain-AI/blob/main/data-collection-and-preprocessing/preprocessed.zip
!wget https://github.com/hs-pm/PulseChain-AI/raw/main/data-collection-and-preprocessing/preprocessed.zip

--2025-06-19 07:03:51--  https://github.com/hs-pm/PulseChain-AI/raw/main/data-collection-and-preprocessing/preprocessed.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hs-pm/PulseChain-AI/main/data-collection-and-preprocessing/preprocessed.zip [following]
--2025-06-19 07:03:51--  https://raw.githubusercontent.com/hs-pm/PulseChain-AI/main/data-collection-and-preprocessing/preprocessed.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3343703 (3.2M) [application/zip]
Saving to: ‘preprocessed.zip’

preprocessed.zip    100%[===================>]   3.19M  --.-KB/s    in 0.06s   

2025-06-19 07:

In [2]:

!unzip /content/preprocessed.zip

Archive:  /content/preprocessed.zip
  inflating: preprocessed_token_terminal_active_addresses_daily.csv  
  inflating: preprocessed_token_terminal_unique_transacting_wallets.csv  
  inflating: preprocessed_token_terminal_transaction_count.csv  
  inflating: preprocessed_token_terminal_total_transaction_fees.csv  
  inflating: preprocessed_token_terminal_market_cap_circulating.csv  
  inflating: preprocessed_token_terminal_transaction_volume.csv  
  inflating: preprocessed_token_terminal_gas_used.csv  
  inflating: preprocessed_token_terminal_tokenholder_revenue.csv  
  inflating: preprocessed_token_terminal_token_incentives.csv  
  inflating: preprocessed_token_terminal_user_dau.csv  
  inflating: preprocessed_token_terminal_expenses.csv  
  inflating: preprocessed_token_terminal_user_wau.csv  
  inflating: preprocessed_token_terminal_gross_profit.csv  
  inflating: preprocessed_token_terminal_market_cap_fully_diluted.csv  
  inflating: preprocessed_token_terminal_active_developers.csv

In [3]:
# prompt: read all csv files and print their name along with their columns

import glob
import pandas as pd

csv_files = glob.glob('*.csv')

for csv_file in csv_files:
    print(f"File: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        print("Columns:", df.columns.tolist())
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")
    print("-" * 20)

File: preprocessed_token_terminal_gas_used.csv
Columns: ['project_id', 'year_month', 'value']
--------------------
File: preprocessed_token_terminal_trade_count.csv
Columns: ['project_id', 'year_month', 'value']
--------------------
File: preprocessed_token_terminal_treasury.csv
Columns: ['project_id', 'year_month', 'timestamp', 'project_name', 'metric_id', 'value']
--------------------
File: preprocessed_token_terminal_user_dau.csv
Columns: ['project_id', 'year_month', 'timestamp', 'project_name', 'metric_id', 'value']
--------------------
File: preprocessed_token_terminal_tvl.csv
Columns: ['project_id', 'year_month', 'timestamp', 'project_name', 'metric_id', 'value']
--------------------
File: preprocessed_token_terminal_active_addresses_daily.csv
Columns: ['project_id', 'year_month', 'timestamp', 'project_name', 'metric_id', 'value']
--------------------
File: preprocessed_token_terminal_earnings.csv
Columns: ['project_id', 'year_month', 'value']
--------------------
File: preproces

In [ ]:
import pandas as pd
import numpy as np
import os

# Directory containing preprocessed files
folder = "/content"

# Load all relevant files into a dictionary
def load_csv(metric_name):
    path = os.path.join(folder, f"preprocessed_token_terminal_{metric_name}.csv")
    print(f"📥 Loading {metric_name} from {path}")
    return pd.read_csv(path)

# Load required dataframes
metric_names = [
    "market_cap_circulating", "market_cap_fully_diluted", "revenue", "tokenholders",
    "user_dau", "user_mau", "fees", "fees_supply_side", "transaction_volume",
    "gross_profit", "cost_of_revenue", "gas_used", "transaction_count",
    "total_transaction_fees", "trading_volume"
]

dfs = {name: load_csv(name) for name in metric_names}

# Clean function
def clean(df):
    return df[["project_id", "year_month", "value"]]

# Clean all DataFrames
for k in dfs:
    dfs[k] = clean(dfs[k])
    print(f"✅ Cleaned: {k}, shape = {dfs[k].shape}")

# Join helper
def merge(left, right, suffixes=("", "_r")):
    print(f"🔗 Merging: {left.shape} + {right.shape}")
    return pd.merge(left, right, on=["project_id", "year_month"], how="inner", suffixes=suffixes)

# Derived Metrics
derived = {}

# 1. Circulating P/S Ratio
df = merge(dfs["market_cap_circulating"], dfs["revenue"])
df["value"] = df["value"] / df["value_r"].replace(0, np.nan)
derived["circulating_ps_ratio"] = df

# 2. Fully Diluted P/S Ratio
derived["fdv_ps_ratio"] = merge(dfs["market_cap_fully_diluted"], dfs["revenue"])
derived["fdv_ps_ratio"]["value"] = derived["fdv_ps_ratio"]["value"] / derived["fdv_ps_ratio"]["value_r"]
print("📊 Derived: fdv_ps_ratio")

# 3. Tokenholder Growth (monthly % growth)
df_tok = dfs["tokenholders"].sort_values(by=["project_id", "year_month"])
df_tok["value_prev"] = df_tok.groupby("project_id")["value"].shift(1)
df_tok["value"] = (df_tok["value"] - df_tok["value_prev"]) / df_tok["value_prev"]
derived["tokenholder_growth"] = df_tok.drop(columns=["value_prev"])
print("📊 Derived: tokenholder_growth")

# 4. DAU / MAU
derived["dau_mau_ratio"] = merge(dfs["user_dau"], dfs["user_mau"])
derived["dau_mau_ratio"]["value"] = derived["dau_mau_ratio"]["value"] / derived["dau_mau_ratio"]["value_r"]
print("📊 Derived: dau_mau_ratio")

# 5. Protocol Take Rate = (fees - supply_side_fees) / transaction_volume
df_take = merge(dfs["fees"], dfs["fees_supply_side"])
df_take["net_fees"] = df_take["value"] - df_take["value_r"]

df_take = merge(df_take, dfs["transaction_volume"], suffixes=("", "_tx"))
df_take["value"] = df_take["net_fees"] / df_take["value_tx"].replace(0, np.nan)

num_missing = df_take["value"].isna().sum()
print(f"📊 Derived: protocol_take_rate — {num_missing} rows with NaN (likely due to 0 transaction_volume)")
derived["protocol_take_rate"] = df_take[["project_id", "year_month", "value"]]

# 6. Gross Margin = gross_profit / revenue
derived["gross_margin"] = merge(dfs["gross_profit"], dfs["revenue"])
derived["gross_margin"]["value"] = derived["gross_margin"]["value"] / derived["gross_margin"]["value_r"]
print("📊 Derived: gross_margin")

# 7. Net Revenue = revenue - cost_of_revenue
derived["net_revenue"] = merge(dfs["revenue"], dfs["cost_of_revenue"])
derived["net_revenue"]["value"] = derived["net_revenue"]["value"] - derived["net_revenue"]["value_r"]
print("📊 Derived: net_revenue")

# 8. Gas Used per Transaction
derived["gas_per_tx"] = merge(dfs["gas_used"], dfs["transaction_count"])
derived["gas_per_tx"]["gas_per_tx"] = (
    derived["gas_per_tx"]["value"] / derived["gas_per_tx"]["value_r"].replace(0, np.nan)
)
print("📊 Derived: gas_per_tx")

# 9. Fees per Transaction
derived["fees_per_tx"] = merge(dfs["fees"], dfs["transaction_count"])
derived["fees_per_tx"]["fees_per_tx"] = (
    derived["fees_per_tx"]["value"] / derived["fees_per_tx"]["value_r"].replace(0, np.nan)
)
print("📊 Derived: fees_per_tx")

# 10. Transaction Volume per User (MAU)
derived["volume_per_user"] = merge(dfs["transaction_volume"], dfs["user_mau"])
derived["volume_per_user"]["value"] = derived["volume_per_user"]["value"] / derived["volume_per_user"]["value_r"].replace(0, np.nan)
print("📊 Derived: volume_per_user")

# Save all derived metrics
output_folder = "derived_metrics"
os.makedirs(output_folder, exist_ok=True)

for metric, df in derived.items():
    df_out = df[["project_id", "year_month", "value"]]
    path = os.path.join(output_folder, f"{metric}.csv")
    df_out.to_csv(path, index=False)
    print(f"✅ Saved: {path}, shape = {df_out.shape}")


📥 Loading market_cap_circulating from /content/preprocessed_token_terminal_market_cap_circulating.csv
📥 Loading market_cap_fully_diluted from /content/preprocessed_token_terminal_market_cap_fully_diluted.csv
📥 Loading revenue from /content/preprocessed_token_terminal_revenue.csv
📥 Loading tokenholders from /content/preprocessed_token_terminal_tokenholders.csv
📥 Loading user_dau from /content/preprocessed_token_terminal_user_dau.csv
📥 Loading user_mau from /content/preprocessed_token_terminal_user_mau.csv
📥 Loading fees from /content/preprocessed_token_terminal_fees.csv
📥 Loading fees_supply_side from /content/preprocessed_token_terminal_fees_supply_side.csv
📥 Loading transaction_volume from /content/preprocessed_token_terminal_transaction_volume.csv
📥 Loading gross_profit from /content/preprocessed_token_terminal_gross_profit.csv
📥 Loading cost_of_revenue from /content/preprocessed_token_terminal_cost_of_revenue.csv
📥 Loading gas_used from /content/preprocessed_token_terminal_gas_used.

In [4]:
import pandas as pd
import numpy as np
import os

# Directory containing preprocessed files
folder = "/content"

# Load all relevant files into a dictionary
def load_csv(metric_name):
    path = os.path.join(folder, f"preprocessed_token_terminal_{metric_name}.csv")
    print(f"📥 Loading {metric_name} from {path}")
    return pd.read_csv(path)

# Load required dataframes
metric_names = [
    "market_cap_circulating", "market_cap_fully_diluted", "revenue", "tokenholders",
    "user_dau", "user_mau", "fees", "fees_supply_side", "transaction_volume",
    "gross_profit", "cost_of_revenue", "gas_used", "transaction_count",
    "total_transaction_fees", "trading_volume"
]

dfs = {name: load_csv(name) for name in metric_names}

# Clean function
def clean(df):
    return df[["project_id", "year_month", "value"]]

# Clean all DataFrames
for k in dfs:
    dfs[k] = clean(dfs[k])
    print(f"✅ Cleaned: {k}, shape = {dfs[k].shape}")

# Join helper
def merge(left, right, suffixes=("", "_r")):
    print(f"🔗 Merging: {left.shape} + {right.shape}")
    return pd.merge(left, right, on=["project_id", "year_month"], how="inner", suffixes=suffixes)

# Derived Metrics
derived = {}

# 1. Circulating P/S Ratio
df = merge(dfs["market_cap_circulating"], dfs["revenue"])
df["value"] = df["value"] / df["value_r"].replace(0, np.nan)
derived["circulating_ps_ratio"] = df

# 2. Fully Diluted P/S Ratio
derived["fdv_ps_ratio"] = merge(dfs["market_cap_fully_diluted"], dfs["revenue"])
derived["fdv_ps_ratio"]["value"] = derived["fdv_ps_ratio"]["value"] / derived["fdv_ps_ratio"]["value_r"]
print("📊 Derived: fdv_ps_ratio")

# 3. Tokenholder Growth
df_tok = dfs["tokenholders"].sort_values(by=["project_id", "year_month"])
df_tok["value_prev"] = df_tok.groupby("project_id")["value"].shift(1)
df_tok["value"] = (df_tok["value"] - df_tok["value_prev"]) / df_tok["value_prev"]
derived["tokenholder_growth"] = df_tok.drop(columns=["value_prev"])
print("📊 Derived: tokenholder_growth")

# 4. DAU / MAU
derived["dau_mau_ratio"] = merge(dfs["user_dau"], dfs["user_mau"])
derived["dau_mau_ratio"]["value"] = derived["dau_mau_ratio"]["value"] / derived["dau_mau_ratio"]["value_r"]
print("📊 Derived: dau_mau_ratio")

# 5. Protocol Take Rate
df_take = merge(dfs["fees"], dfs["fees_supply_side"])
df_take["net_fees"] = df_take["value"] - df_take["value_r"]
df_take = merge(df_take, dfs["transaction_volume"], suffixes=("", "_tx"))
df_take["value"] = df_take["net_fees"] / df_take["value_tx"].replace(0, np.nan)
derived["protocol_take_rate"] = df_take[["project_id", "year_month", "value"]]
print(f"📊 Derived: protocol_take_rate — {df_take['value'].isna().sum()} NaNs")

# 6. Gross Margin
derived["gross_margin"] = merge(dfs["gross_profit"], dfs["revenue"])
derived["gross_margin"]["value"] = derived["gross_margin"]["value"] / derived["gross_margin"]["value_r"]
print("📊 Derived: gross_margin")

# 7. Net Revenue
derived["net_revenue"] = merge(dfs["revenue"], dfs["cost_of_revenue"])
derived["net_revenue"]["value"] = derived["net_revenue"]["value"] - derived["net_revenue"]["value_r"]
print("📊 Derived: net_revenue")

# 8. Gas Used per Transaction
derived["gas_per_tx"] = merge(dfs["gas_used"], dfs["transaction_count"])
derived["gas_per_tx"]["gas_per_tx"] = derived["gas_per_tx"]["value"] / derived["gas_per_tx"]["value_r"].replace(0, np.nan)
print("📊 Derived: gas_per_tx")

# 9. Fees per Transaction
derived["fees_per_tx"] = merge(dfs["fees"], dfs["transaction_count"])
derived["fees_per_tx"]["fees_per_tx"] = derived["fees_per_tx"]["value"] / derived["fees_per_tx"]["value_r"].replace(0, np.nan)
print("📊 Derived: fees_per_tx")

# 10. Transaction Volume per User (MAU)
derived["volume_per_user"] = merge(dfs["transaction_volume"], dfs["user_mau"])
derived["volume_per_user"]["value"] = derived["volume_per_user"]["value"] / derived["volume_per_user"]["value_r"].replace(0, np.nan)
print("📊 Derived: volume_per_user")







# Load and filter price data
price_df = load_csv("price")

# Ensure we're only working with 'price' metric
price_df = price_df[price_df["metric_id"] == "price"]

# Use only necessary columns and sort
price_df = price_df[["project_id", "timestamp", "value"]].copy()
price_df["timestamp"] = pd.to_datetime(price_df["timestamp"])
price_df = price_df.sort_values(["project_id", "timestamp"])

# Calculate metrics
price_df["momentum_30d"] = price_df.groupby("project_id")["value"].pct_change(periods=30)
price_df["log_return"] = np.log(price_df["value"] / price_df["value"].shift(1))
price_df["volatility_30d"] = price_df.groupby("project_id")["log_return"].transform(lambda x: x.rolling(30).std())
price_df["sharpe_like_ratio"] = price_df["momentum_30d"] / price_df["volatility_30d"]

# Max Drawdown
def max_drawdown(prices):
    roll_max = prices.rolling(90, min_periods=1).max()
    drawdown = prices / roll_max - 1.0
    return drawdown.rolling(90, min_periods=1).min()

price_df["drawdown_max_90d"] = price_df.groupby("project_id")["value"].transform(max_drawdown)

# Correlation with BTC
btc_df = price_df[price_df["project_id"] == "bitcoin"][["timestamp", "value"]].rename(columns={"value": "btc_price"})
price_df = price_df.merge(btc_df, on="timestamp", how="left")

def rolling_corr(x):
    return x["value"].rolling(30).corr(x["btc_price"])

price_df["correlation_with_btc"] = price_df.groupby("project_id").apply(rolling_corr).reset_index(level=0, drop=True)

# Add year_month for monthly aggregation
price_df["year_month"] = price_df["timestamp"].dt.to_period("M").astype(str)

# Store in derived dictionary
metrics_to_save = {
    "price_momentum_30d": "momentum_30d",
    "price_volatility_30d": "volatility_30d",
    "sharpe_like_ratio": "sharpe_like_ratio",
    "price_drawdown_max_90d": "drawdown_max_90d",
    "correlation_with_btc": "correlation_with_btc",
}

for metric_name, col in metrics_to_save.items():
    df_out = price_df[["project_id", "year_month", col]].dropna().rename(columns={col: "value"})
    derived[metric_name] = df_out
    print(f"📊 Derived: {metric_name}")

# Save all derived metrics
output_folder = "derived_metrics"
os.makedirs(output_folder, exist_ok=True)

for metric, df in derived.items():
    df_out = df[["project_id", "year_month", "value"]]
    path = os.path.join(output_folder, f"{metric}.csv")
    df_out.to_csv(path, index=False)
    print(f"✅ Saved: {path}, shape = {df_out.shape}")


📥 Loading market_cap_circulating from /content/preprocessed_token_terminal_market_cap_circulating.csv
📥 Loading market_cap_fully_diluted from /content/preprocessed_token_terminal_market_cap_fully_diluted.csv
📥 Loading revenue from /content/preprocessed_token_terminal_revenue.csv
📥 Loading tokenholders from /content/preprocessed_token_terminal_tokenholders.csv
📥 Loading user_dau from /content/preprocessed_token_terminal_user_dau.csv
📥 Loading user_mau from /content/preprocessed_token_terminal_user_mau.csv
📥 Loading fees from /content/preprocessed_token_terminal_fees.csv
📥 Loading fees_supply_side from /content/preprocessed_token_terminal_fees_supply_side.csv
📥 Loading transaction_volume from /content/preprocessed_token_terminal_transaction_volume.csv
📥 Loading gross_profit from /content/preprocessed_token_terminal_gross_profit.csv
📥 Loading cost_of_revenue from /content/preprocessed_token_terminal_cost_of_revenue.csv
📥 Loading gas_used from /content/preprocessed_token_terminal_gas_used.

/tmp/ipython-input-4-2001614143.py:134: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  price_df["correlation_with_btc"] = price_df.groupby("project_id").apply(rolling_corr).reset_index(level=0, drop=True)


📊 Derived: price_momentum_30d
📊 Derived: price_volatility_30d
📊 Derived: sharpe_like_ratio
📊 Derived: price_drawdown_max_90d
📊 Derived: correlation_with_btc
✅ Saved: derived_metrics/circulating_ps_ratio.csv, shape = (7575, 3)
✅ Saved: derived_metrics/fdv_ps_ratio.csv, shape = (7283, 3)
✅ Saved: derived_metrics/tokenholder_growth.csv, shape = (13557, 3)
✅ Saved: derived_metrics/dau_mau_ratio.csv, shape = (9972, 3)
✅ Saved: derived_metrics/protocol_take_rate.csv, shape = (164, 3)
✅ Saved: derived_metrics/gross_margin.csv, shape = (497, 3)
✅ Saved: derived_metrics/net_revenue.csv, shape = (439, 3)
✅ Saved: derived_metrics/gas_per_tx.csv, shape = (94, 3)
✅ Saved: derived_metrics/fees_per_tx.csv, shape = (1773, 3)
✅ Saved: derived_metrics/volume_per_user.csv, shape = (192, 3)
✅ Saved: derived_metrics/price_momentum_30d.csv, shape = (7229, 3)
✅ Saved: derived_metrics/price_volatility_30d.csv, shape = (7513, 3)
✅ Saved: derived_metrics/sharpe_like_ratio.csv, shape = (7229, 3)
✅ Saved: derived

In [ ]:
import random

for metric_name, df in derived.items():
    print(f"\n📈 Metric: {metric_name}           ***************")

    # Drop NaN values to avoid selecting projects with all nulls
    non_null_df = df.dropna(subset=["value"])

    # Get list of unique project_ids with at least one non-null value
    projects = non_null_df["project_id"].unique()

    if len(projects) < 3:
        print(f"⚠️ Not enough non-null projects to sample from: {len(projects)} found.")
        continue

    # Pick 3 random projects
    sampled_projects = random.sample(list(projects), 3)

    for project_id in sampled_projects:
        print(f"\n🔹 Project: {project_id}")
        print(non_null_df[non_null_df["project_id"] == project_id].tail(3))



📈 Metric: circulating_ps_ratio           ***************

🔹 Project: apeswap
    project_id year_month      value      value_r
530    apeswap    2025-01  19.650687  8281.628317
531    apeswap    2025-02  19.957028  7012.766774
532    apeswap    2025-03  28.343347  4632.294888

🔹 Project: cap
     project_id year_month       value      value_r
1555        cap    2024-11  298.889184   360.006765
1556        cap    2024-12  590.263270  1467.268343
1557        cap    2025-01    7.604536   699.968581

🔹 Project: instadapp
     project_id year_month       value        value_r
3454  instadapp    2025-04  544.090652  335771.803152
3455  instadapp    2025-05  308.564546  528767.898113
3456  instadapp    2025-06  637.927353  248890.835779

📈 Metric: fdv_ps_ratio           ***************

🔹 Project: peaq
     project_id year_month          value      value_r
4818       peaq    2025-04   89854.847148  6151.943161
4819       peaq    2025-05  126697.730522  4583.740630
4820       peaq    2025-06  

In [ ]:
# prompt: find derived rows where project_id=bitcoin and show for volume_per_user

print("\nBitcoin Volume per User:")
print(derived["volume_per_user"][derived["volume_per_user"]["project_id"] == "bitcoin"].tail(3))
print(dfs["transaction_volume"][dfs["transaction_volume"]["project_id"] == "bitcoin"].tail(2))
print(dfs["user_mau"][dfs["user_mau"]["project_id"] == "bitcoin"].tail(2))


Bitcoin Volume per User:
Empty DataFrame
Columns: [project_id, year_month, value, value_r]
Index: []
Empty DataFrame
Columns: [project_id, year_month, value]
Index: []
     project_id year_month     value
1656    bitcoin    2025-05  10844063
1657    bitcoin    2025-06  10918829


In [ ]:
# # prompt: read /content/derived_metrics folder
# # find out all csvs
# # read all csvs
# # find out project_ids having at least 1  non-null entry and save them in list for each csv
# # use the non-null list to randomly select 3 project_ids for each csv
# # print df.tail for each of the selected project_ids

# import pandas as pd
# derived_csv_files = glob.glob(os.path.join("derived_metrics", "*.csv"))

# for csv_file_path in derived_csv_files:
#     print(f"\n--- Processing File: {os.path.basename(csv_file_path)} ---")
#     try:
#         df = pd.read_csv(csv_file_path)

#         # Find project_ids with at least one non-null value in the 'value' column
#         non_null_projects = df.dropna(subset=['value'])['project_id'].unique().tolist()

#         if len(non_null_projects) == 0:
#             print("No projects with non-null values found.")
#             continue

#         print(f"Found {len(non_null_projects)} projects with non-null values.")

#         # Randomly select up to 3 project_ids (or fewer if less than 3 available)
#         num_to_sample = min(3, len(non_null_projects))
#         sampled_project_ids = random.sample(non_null_projects, num_to_sample)

#         print(f"Randomly selected {num_to_sample} projects: {sampled_project_ids}")

#         # Print df.tail for each selected project_id
#         for project_id in sampled_project_ids:
#             print(f"\nTail of data for project_id: {project_id}")
#             project_df = df[df['project_id'] == project_id].dropna(subset=['value'])
#             if not project_df.empty:
#                 print(project_df.tail())
#             else:
#                 print("No non-null data points found for this project_id.")

#     except Exception as e:
#         print(f"Error processing {csv_file_path}: {e}")
#     print("-" * 30)

In [ ]:
# # prompt: print latest gas_fee for all project_ids in derived

# # Select the gas_per_tx DataFrame
# gas_per_tx_df = derived["gas_per_tx"].copy()

# # Sort by project and year_month to get the latest entry easily
# gas_per_tx_df = gas_per_tx_df.sort_values(by=["project_id", "year_month"])

# # Get the latest entry for each project_id
# latest_gas_fee = gas_per_tx_df.groupby("project_id").tail(1)

# print("\nLatest Gas Used per Transaction (gas_per_tx) for all project_ids:")
# latest_gas_fee

In [ ]:
# # prompt: print latest gas_fee for all project_ids in derived and print all project_ids for which there is even 1 non null non zero gas_fee

# # Find the gas_fees DataFrame in the dfs dictionary
# gas_fees_df = dfs.get("fees")

# if gas_fees_df is not None:
#     print("\n--- Latest Gas Fees for all project_ids ---")
#     # Sort by project_id and year_month to easily get the latest entry for each project
#     gas_fees_df_sorted = gas_fees_df.sort_values(by=['project_id', 'year_month'])

#     # Get the last entry for each project_id
#     latest_gas_fees = gas_fees_df_sorted.groupby('project_id').tail(1)

#     # Print the latest gas fees
#     if not latest_gas_fees.empty:
#         print(latest_gas_fees[['project_id', 'year_month', 'value']])
#     else:
#         print("No gas fees data found.")

#     print("\n--- Project_ids with at least one non-null, non-zero gas fee ---")
#     # Filter for rows where 'value' is not null and not zero
#     non_null_zero_gas_fees = gas_fees_df[(gas_fees_df['value'].notna()) & (gas_fees_df['value'] != 0)]

#     # Get unique project_ids from the filtered DataFrame
#     projects_with_non_null_zero_gas_fees = non_null_zero_gas_fees['project_id'].unique().tolist()

#     if projects_with_non_null_zero_gas_fees:
#         print(projects_with_non_null_zero_gas_fees)
#     else:
#         print("No project_ids found with a non-null, non-zero gas fee.")
# else:
#     print("Gas fees DataFrame not found in the loaded data.")

In [ ]:

# uniswap_fees_per_tx = derived["fees_per_tx"][derived["fees_per_tx"]["project_id"] == "uniswap"]

# print("\nUniswap Fees per Transaction (fees_per_tx):")
# if not uniswap_fees_per_tx.empty:
#     print(uniswap_fees_per_tx.tail())
# else:
#     print("No fees_per_tx data found for Uniswap.")

#     # prompt: from derived show gas_fees, transaction_count for uniswap

# # Filter derived gas_per_tx for 'uniswap'
# uniswap_gas_per_tx = derived["gas_per_tx"][derived["gas_per_tx"]["project_id"] == "uniswap"]

# # Filter dfs transaction_count for 'uniswap'
# uniswap_transaction_count = dfs["transaction_count"][dfs["transaction_count"]["project_id"] == "uniswap"]

# print("\nUniswap Gas Used per Transaction (gas_per_tx):")
# if not uniswap_gas_per_tx.empty:
#     print(uniswap_gas_per_tx.tail())
# else:
#     print("No gas_per_tx data found for Uniswap.")

# print("\nUniswap Transaction Count:")
# if not uniswap_transaction_count.empty:
#     print(uniswap_transaction_count.tail())
# else:
#     print("No transaction_count data found for Uniswap.")

# # Find the gas_fees DataFrame in the dfs dictionary and filter for 'uniswap'
# uniswap_gas_fees = dfs.get("fees")
# if uniswap_gas_fees is not None:
#     uniswap_gas_fees = uniswap_gas_fees[uniswap_gas_fees["project_id"] == "uniswap"]
#     print("\nUniswap Gas Fees (derived from 'fees'):")
#     if not uniswap_gas_fees.empty:
#         print(uniswap_gas_fees.tail())
#     else:
#         print("No 'fees' data found for Uniswap.")
# else:
#     print("Gas fees DataFrame (derived from 'fees') not found in the loaded data.")


In [ ]:
# # prompt: dfs["market_cap_fully_diluted"] for kwenta project_id

# kwenta_market_cap_fd = dfs["market_cap_fully_diluted"][dfs["market_cap_fully_diluted"]["project_id"] == "kwenta"]

# print("\nKwenta Market Cap Fully Diluted:")
# if not kwenta_market_cap_fd.empty:
#     print(kwenta_market_cap_fd.tail())
# else:
#     print("No 'market_cap_fully_diluted' data found for Kwenta.")
# # prompt: dfs["market_cap_fully_diluted"] for kwenta project_id

# kwenta_market_cap_fd = dfs["revenue"][dfs["revenue"]["project_id"] == "kwenta"]

# print("\nKwenta revenue Fully Diluted:")
# if not kwenta_market_cap_fd.empty:
#     print(kwenta_market_cap_fd.tail())
# else:
#     print("No 'revenue' data found for Kwenta.")

In [5]:
# prompt: zip derived_metrics

!zip -r derived_metrics.zip derived_metrics

  adding: derived_metrics/ (stored 0%)
  adding: derived_metrics/gross_margin.csv (deflated 68%)
  adding: derived_metrics/protocol_take_rate.csv (deflated 72%)
  adding: derived_metrics/gas_per_tx.csv (deflated 64%)
  adding: derived_metrics/tokenholder_growth.csv (deflated 67%)
  adding: derived_metrics/price_drawdown_max_90d.csv (deflated 84%)
  adding: derived_metrics/price_volatility_30d.csv (deflated 66%)
  adding: derived_metrics/price_momentum_30d.csv (deflated 65%)
  adding: derived_metrics/fees_per_tx.csv (deflated 63%)
  adding: derived_metrics/correlation_with_btc.csv (deflated 66%)
  adding: derived_metrics/fdv_ps_ratio.csv (deflated 66%)
  adding: derived_metrics/circulating_ps_ratio.csv (deflated 66%)
  adding: derived_metrics/sharpe_like_ratio.csv (deflated 64%)
  adding: derived_metrics/volume_per_user.csv (deflated 67%)
  adding: derived_metrics/net_revenue.csv (deflated 63%)
  adding: derived_metrics/dau_mau_ratio.csv (deflated 69%)


In [6]:
import requests

url = "https://api.coingecko.com/api/v3/global"
response = requests.get(url).json()

btc_dominance = response["data"]["market_cap_percentage"]["btc"]
eth_dominance = response["data"]["market_cap_percentage"]["eth"]
global_volume_usd = response["data"]["total_volume"]["usd"]

print(f"BTC Dominance: {btc_dominance:.2f}%")
print(f"ETH Dominance: {eth_dominance:.2f}%")
print(f"Global Trading Volume: ${global_volume_usd:,.0f}")


BTC Dominance: 61.74%
ETH Dominance: 9.03%
Global Trading Volume: $100,488,720,752


In [11]:
import requests

# Example historical date — MUST be in dd-mm-yyyy format
date_str = "02-01-2021"

url = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={date_str}&localization=false"
resp = requests.get(url)

if resp.status_code == 200:
    data = resp.json()
    if "market_data" in data:
        usd_market_cap = data["market_data"]["market_cap"].get("usd")
        print(f"✅ Bitcoin Market Cap on {date_str}: ${usd_market_cap:,.0f}")
    else:
        print(f"❌ No market_data for Bitcoin on {date_str}")
else:
    print(f"❌ HTTP error {resp.status_code}")


❌ HTTP error 401


In [28]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# ---------- Safe request wrapper with 429 handling ----------

def safe_request(url, coin_id, date_str, label):
    headers = {"User-Agent": "Mozilla/5.0"}
    while True:
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            return resp
        elif resp.status_code == 429:
            print(f"   ⏳ Rate limited (429) for {label} on {date_str}. Waiting 60s...")
            time.sleep(60)
        else:
            print(f"   ❌ HTTP error {resp.status_code} for {label} on {date_str}")
            return None

# ---------- Helpers ----------

def get_coin_marketcap_on_date(coin_id, date_str):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/history?date={date_str}&localization=false"
    resp = safe_request(url, coin_id, date_str, f"{coin_id} cap")
    if resp is None:
        return None

    data = resp.json()
    if "market_data" not in data:
        print(f"   ❌ No market_data for {coin_id} on {date_str}")
        return None

    try:
        return data["market_data"]["market_cap"]["usd"]
    except KeyError:
        print(f"   ❌ Missing market_cap['usd'] for {coin_id} on {date_str}")
        return None


def get_global_volume_on_date(date_str):
    url = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={date_str}&localization=false"
    resp = safe_request(url, "bitcoin", date_str, "global volume")
    if resp is None:
        return None

    data = resp.json()
    if "market_data" not in data:
        print(f"   ❌ No market_data for volume on {date_str}")
        return None

    return data["market_data"]["total_volume"].get("usd", None)

# ---------- Date Setup ----------

today = datetime.today()
one_year_ago = today - timedelta(days=365)

# Create month start dates from the last 3 months
month_starts = pd.date_range(start=one_year_ago.replace(day=1), end=today, freq="MS")

# ---------- Retry Logic Per Month ----------

def try_fetch_market_data(month_start):
    for offset in range(3):  # Try 1st, 2nd, 3rd
        try_date = month_start + timedelta(days=offset)
        date_str = try_date.strftime("%d-%m-%Y")
        print(f"📆 Trying {date_str}...")

        time.sleep(10)
        print(f"   🔍 Fetching BTC market cap...")
        btc_cap = get_coin_marketcap_on_date("bitcoin", date_str)

        time.sleep(10)
        print(f"   🔍 Fetching ETH market cap...")
        eth_cap = get_coin_marketcap_on_date("ethereum", date_str)

        time.sleep(10)
        print(f"   🔍 Fetching global volume...")
        global_vol = get_global_volume_on_date(date_str)

        if btc_cap is not None and eth_cap is not None:
            total_cap = btc_cap + eth_cap
            return {
                "year_month": try_date.strftime("%Y-%m"),
                "timestamp": try_date,
                "btc_dominance": btc_cap / total_cap * 100,
                "eth_dominance": eth_cap / total_cap * 100,
                "global_trading_volume": global_vol,
            }

    print(f"⚠️  Failed to get data for {month_start.strftime('%Y-%m')} after 3 attempts.")
    return None

# ---------- Fetch All Months ----------

records = []
for month_start in month_starts:
    record = try_fetch_market_data(month_start)
    if record:
        records.append(record)

# ---------- Format and Save ----------

if records:
    df = pd.DataFrame(records)
    df["project_id"] = "global"

    df_long = df.melt(
        id_vars=["project_id", "year_month", "timestamp"],
        value_vars=["btc_dominance", "eth_dominance", "global_trading_volume"],
        var_name="metric_id",
        value_name="value"
    )

    df_long = df_long[["project_id", "year_month", "timestamp", "metric_id", "value"]]
    df_long.to_csv("derived_metrics/crypto_market_indicators_monthly.csv", index=False)

    print(f"✅ Saved: derived_metrics/crypto_market_indicators_monthly.csv")
    print(f"📊 Shape: {df_long.shape}")
else:
    print("⚠ No data fetched — all months failed.")


📆 Trying 01-06-2024...
   🔍 Fetching BTC market cap...
   ❌ HTTP error 401 for bitcoin cap on 01-06-2024
   🔍 Fetching ETH market cap...
   ❌ HTTP error 401 for ethereum cap on 01-06-2024
   🔍 Fetching global volume...
   ❌ HTTP error 401 for global volume on 01-06-2024
📆 Trying 02-06-2024...
   🔍 Fetching BTC market cap...
   ❌ HTTP error 401 for bitcoin cap on 02-06-2024
   🔍 Fetching ETH market cap...
   ❌ HTTP error 401 for ethereum cap on 02-06-2024
   🔍 Fetching global volume...
   ❌ HTTP error 401 for global volume on 02-06-2024
📆 Trying 03-06-2024...
   🔍 Fetching BTC market cap...
   ❌ HTTP error 401 for bitcoin cap on 03-06-2024
   🔍 Fetching ETH market cap...
   ❌ HTTP error 401 for ethereum cap on 03-06-2024
   🔍 Fetching global volume...
   ❌ HTTP error 401 for global volume on 03-06-2024
⚠️  Failed to get data for 2024-06 after 3 attempts.
📆 Trying 01-07-2024...
   🔍 Fetching BTC market cap...
   🔍 Fetching ETH market cap...
   🔍 Fetching global volume...
   ⏳ Rate limite

In [31]:
df_long.to_csv('global_data.csv')

In [36]:
import requests
import pandas as pd

# Correct endpoint
url = "https://api.llama.fi/v2/historicalChainTvl"
response = requests.get(url)

# Ensure success
if response.status_code != 200:
    raise Exception(f"Request failed with status {response.status_code}")

# Parse response
data = response.json()  # this is a list of dicts

# Convert to DataFrame
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'], unit='s')

# Group by date to sum TVL across chains
df_total = df.groupby('date')['tvl'].sum().resample('M').last().reset_index()
df_total['metric_id'] = 'total_tvl_usd'
df_total.rename(columns={'tvl': 'value'}, inplace=True)

# Preview
print(df_total.tail())


         date         value      metric_id
82 2025-02-28   98129682473  total_tvl_usd
83 2025-03-31   91634365969  total_tvl_usd
84 2025-04-30   98712801004  total_tvl_usd
85 2025-05-31  112692682454  total_tvl_usd
86 2025-06-30  112822459224  total_tvl_usd


/tmp/ipython-input-36-1988610659.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_total = df.groupby('date')['tvl'].sum().resample('M').last().reset_index()


In [64]:
import yfinance as yf
import pandas as pd

# Download S&P 500 data
sp500 = yf.download('^GSPC', start="2023-01-01", interval="1d", group_by='ticker')

# Flatten MultiIndex columns if present
if isinstance(sp500.columns, pd.MultiIndex):
    sp500.columns = sp500.columns.get_level_values(0)

# Remove column names and index name
sp500.columns.name = None
sp500.index.name = None

# Confirm: Get first few rows
print(sp500.head())

print(sp500.columns)
sp500.columns=['sp500_index','a','b','c','d']
sp500=sp500['sp500_index']
sp500 = sp500.reset_index()

# Step 2: Rename the index column to 'month'
sp500.rename(columns={'index': 'month'}, inplace=True)

/tmp/ipython-input-64-4105661863.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download('^GSPC', start="2023-01-01", interval="1d", group_by='ticker')
[*********************100%***********************]  1 of 1 completed

                  ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
2023-01-03  3853.290039  3878.459961  3794.330078  3824.139893  3959140000
2023-01-04  3840.360107  3873.159912  3815.770020  3852.969971  4414080000
2023-01-05  3839.739990  3839.739990  3802.419922  3808.100098  3893450000
2023-01-06  3823.370117  3906.189941  3809.560059  3895.080078  3923560000
2023-01-09  3910.820068  3950.570068  3890.419922  3892.090088  4311770000
Index(['^GSPC', '^GSPC', '^GSPC', '^GSPC', '^GSPC'], dtype='object')


In [68]:
sp500.rename(columns={'sp500_index': 'value'}, inplace=True)
sp500['metric_id']='sp500_index'

In [69]:
sp500

,month,value,metric,metric_id
0,2023-01-03,3853.290039,sp500_index,sp500_index
1,2023-01-04,3840.360107,sp500_index,sp500_index
2,2023-01-05,3839.739990,sp500_index,sp500_index
3,2023-01-06,3823.370117,sp500_index,sp500_index
4,2023-01-09,3910.820068,sp500_index,sp500_index
...,...,...,...,...
612,2025-06-12,6009.899902,sp500_index,sp500_index
613,2025-06-13,6000.560059,sp500_index,sp500_index
614,2025-06-16,6004.000000,sp500_index,sp500_index
615,2025-06-17,6012.149902,sp500_index,sp500_index


In [73]:
import pandas as pd

# --- 1. Coingecko dominance and volume ---
df_long['year_month'] = pd.to_datetime(df_long['timestamp']).dt.to_period('M').astype(str)
df_cg = df_long.groupby(['year_month', 'metric_id'])['value'].last().reset_index()

# --- 2. TVL ---
df_total['year_month'] = pd.to_datetime(df_total['date']).dt.to_period('M').astype(str)
df_tvl = df_total.groupby(['year_month', 'metric_id'])['value'].last().reset_index()

# --- 3. S&P 500 ---
df_sp = sp500.copy()
df_sp['year_month'] = pd.to_datetime(df_sp['month']).dt.to_period('M').astype(str)
df_sp = df_sp[['year_month', 'metric_id', 'value']]

# --- 4. Combine all ---
global_market = pd.concat([df_cg, df_tvl, df_sp], ignore_index=True)

# --- 5. Clean and sort ---
global_market = global_market.dropna()
global_market = global_market.sort_values(by=['year_month', 'metric_id'])

# --- 6. Save ---
global_market.to_csv('global_market.csv', index=False)


In [76]:
# prompt: list all csvs in /content/derived_metrics
# and all csvs in /content

import glob
import os

# List CSVs in /content/derived_metrics
print("CSV files in /content/derived_metrics:")
derived_metrics_csvs = glob.glob('/content/derived_metrics/*.csv')
for f in derived_metrics_csvs:
    print(f)

print("\nCSV files in /content:")
# List CSVs in /content
content_csvs = glob.glob('/content/*.csv')
for f in content_csvs:
  print(f)

CSV files in /content/derived_metrics:
/content/derived_metrics/gross_margin.csv
/content/derived_metrics/crypto_market_indicators_monthly.csv
/content/derived_metrics/protocol_take_rate.csv
/content/derived_metrics/gas_per_tx.csv
/content/derived_metrics/tokenholder_growth.csv
/content/derived_metrics/price_drawdown_max_90d.csv
/content/derived_metrics/price_volatility_30d.csv
/content/derived_metrics/price_momentum_30d.csv
/content/derived_metrics/fees_per_tx.csv
/content/derived_metrics/correlation_with_btc.csv
/content/derived_metrics/fdv_ps_ratio.csv
/content/derived_metrics/circulating_ps_ratio.csv
/content/derived_metrics/sharpe_like_ratio.csv
/content/derived_metrics/volume_per_user.csv
/content/derived_metrics/net_revenue.csv
/content/derived_metrics/dau_mau_ratio.csv

CSV files in /content:
/content/preprocessed_token_terminal_gas_used.csv
/content/preprocessed_token_terminal_trade_count.csv
/content/preprocessed_token_terminal_treasury.csv
/content/preprocessed_token_termina